<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/On_God.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
# Install Dependencies (Colab Only)
!pip install deepface streamlit opencv-python-headless

# Import Libraries
import os
import pandas as pd
import numpy as np
from deepface import DeepFace
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Configuration
CONFIG = {
    "known_faces_dir": "/content/drive/MyDrive/known_faces",
    "unknown_faces_dir": "/content/drive/MyDrive/unknown_faces",
    "database_path": "/content/drive/MyDrive/face_embeddings.csv",
    "model_name": "ArcFace",  # Alternatives: "Facenet", "VGG-Face"
    "threshold": 0.55,  # Adjusted for improved matching
    "image_extensions": ('.png', '.jpg', '.jpeg'),
    "detector": "retinaface"  # More accurate than opencv/ssd
}

# Helper Functions
def is_image_file(filename):
    return filename.lower().endswith(CONFIG['image_extensions'])

def clean_directory_listing(path):
    return [f for f in os.listdir(path) if is_image_file(f)]

# Enhanced Embedding Extraction
def extract_embeddings():
    data = []
    print("⏳ Extracting face embeddings...")

    for person_name in os.listdir(CONFIG['known_faces_dir']):
        person_path = os.path.join(CONFIG['known_faces_dir'], person_name)

        if not os.path.isdir(person_path):
            continue

        for img_name in clean_directory_listing(person_path):
            img_path = os.path.join(person_path, img_name)

            try:
                # Get embedding with enhanced detector
                embedding = DeepFace.represent(
                    img_path=img_path,
                    model_name=CONFIG['model_name'],
                    detector_backend=CONFIG['detector'],
                    enforce_detection=True
                )[0]['embedding']

                data.append({
                    "name": person_name,
                    "embedding": np.array(embedding).tolist(),  # Store as list
                    "image_path": img_path
                })
                print(f"✅ Processed: {person_name}/{img_name}")

            except Exception as e:
                print(f"❌ Failed on {img_path}: {str(e)}")
                continue

    if data:
        pd.DataFrame(data).to_csv(CONFIG['database_path'], index=False)
        print(f"💾 Saved {len(data)} embeddings to {CONFIG['database_path']}")
    else:
        print("❌ No valid faces found!")

# Optimized Recognition (without FAISS)
def recognize_faces():
    try:
        df = pd.read_csv(CONFIG['database_path'])
        df['embedding'] = df['embedding'].apply(eval)  # Convert string to list
    except Exception as e:
        print(f"❌ Database error: {e}")
        return

    print("\n🔍 Starting recognition...")

    for img_name in clean_directory_listing(CONFIG['unknown_faces_dir']):
        img_path = os.path.join(CONFIG['unknown_faces_dir'], img_name)

        try:
            # Get query embedding
            query_embedding = DeepFace.represent(
                img_path=img_path,
                model_name=CONFIG['model_name'],
                detector_backend=CONFIG['detector'],
                enforce_detection=True
            )[0]['embedding']
            query_embedding = np.array(query_embedding)

            # Calculate cosine similarities
            similarities = []
            for _, row in df.iterrows():
                known_embedding = np.array(row['embedding'])
                # Cosine similarity = dot product of normalized vectors
                similarity = np.dot(query_embedding, known_embedding) / (
                    np.linalg.norm(query_embedding) * np.linalg.norm(known_embedding)
                )
                similarities.append(similarity)

            best_match_idx = np.argmax(similarities)
            best_similarity = similarities[best_match_idx]
            match_name = df.iloc[best_match_idx]['name']

            print(f"\n📸 Processing: {img_name}")
            print(f"🔎 Best Match: {match_name}")
            print(f"📏 Similarity: {best_similarity:.4f} (Threshold: {CONFIG['threshold']})")

            # Log all similarity scores
            for idx, similarity in enumerate(similarities):
                print(f"ℹ️ {df.iloc[idx]['name']}: {similarity:.4f}")

            if best_similarity > CONFIG['threshold']:
                print(f"✅ VERIFIED: {match_name}")
            else:
                print("❌ UNKNOWN FACE")

        except Exception as e:
            print(f"⚠️ Error processing {img_name}: {str(e)}")

# Execute
print("⭐ Face Recognition System ⭐")
extract_embeddings()
recognize_faces()

# Verification
print("\n🔬 Summary")
print("Known faces:", os.listdir(CONFIG['known_faces_dir']))
print("Tested unknown faces:", clean_directory_listing(CONFIG['unknown_faces_dir']))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⭐ Face Recognition System ⭐
⏳ Extracting face embeddings...
✅ Processed: Swidess/12.jpg
✅ Processed: Silvanus/5.jpg
✅ Processed: Trump/trump 3.jpeg
✅ Processed: Steve/steve1.jpg
💾 Saved 4 embeddings to /content/drive/MyDrive/face_embeddings.csv

🔍 Starting recognition...

📸 Processing: trump 2.jpeg
🔎 Best Match: Trump
📏 Similarity: 0.5698 (Threshold: 0.55)
ℹ️ Swidess: 0.0364
ℹ️ Silvanus: 0.0882
ℹ️ Trump: 0.5698
ℹ️ Steve: 0.0949
✅ VERIFIED: Trump

📸 Processing: steve2.jpg
🔎 Best Match: Steve
📏 Similarity: 0.5861 (Threshold: 0.55)
ℹ️ Swidess: -0.2005
ℹ️ Silvanus: -0.0454
ℹ️ Trump: 0.0014
ℹ️ Steve: 0.5861
✅ VERIFIED: Steve

📸 Processing: 13.jpg
🔎 Best Match: Swidess
📏 Similarity: 0.6768 (Threshold: 0.55)
ℹ️ Swidess: 0.6768
ℹ️ Silvanus: 0.1431
ℹ️ Trump: -0.0050
ℹ️ Steve: -0.0776
✅ VERIFIED: Swidess

🔬 Summary
Known faces: ['Swidess', '.ipynb_checkpoints', 'Silvan

In [76]:
df = pd.read_csv(DATABASE_PATH)
print(df.head())

       name                                          embedding  \
0   Swidess  [-0.120555579662323, 0.16487948596477509, 0.31...   
1  Silvanus  [-0.05616405978798866, 0.264980673789978, 0.01...   
2     Trump  [-0.010562478564679623, -0.08159864693880081, ...   
3     Steve  [0.30614060163497925, -0.24800869822502136, -0...   

                                          image_path  
0  /content/drive/MyDrive/known_faces/Swidess/12.jpg  
1  /content/drive/MyDrive/known_faces/Silvanus/5.jpg  
2  /content/drive/MyDrive/known_faces/Trump/trump...  
3  /content/drive/MyDrive/known_faces/Steve/steve...  
